# Part 5 - Explore the Clusters
In this notebook, I will explore the clusters created by KMeans and create an interactive Folium map of the clusters.  

<div style="text-align: left">
    <h2>
        <a href="https://nbviewer.jupyter.org/github/KathrynDH/IBMCapstoneFinalProject/blob/master/Cluster%20Metro%20Areas.ipynb">&larr; Part 4</a>
    </h2>
</div>


In [1]:
#!conda install -c conda-forge folium=0.5.0 --yes

In [2]:
#import libraries
import requests
import numpy as np
import pandas as pd
import ibm_boto3
import io

In [3]:
from IPython.display import Image 
from IPython.core.display import HTML 
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib as mpl

In [4]:
# The code was removed by Watson Studio for sharing.

In [5]:
#Import dataframe
obj = resource.Object(bucket_name=bucket, key='VenueDataWithClusters.json').get()
df = pd.read_json(io.BytesIO(obj['Body'].read()))
df.sort_index(inplace=True)
df.head()

,Metropolitan,Country,Continent,Latitude,Longitude,Category 1,Category 2,Category 3,Category 4,Category 5,Category 6,Category 7,Category 8,Category 9,Category 10,Cluster Labels
0,Tokyo,Japan,Asia,35.682839,139.759455,Café,Japanese Restaurant,Italian Restaurant,Sushi Restaurant,Lounge,Garden,French Restaurant,Dessert Shop,Thai Restaurant,Steakhouse,9
1,Delhi,India,Asia,28.651718,77.221939,Indian Restaurant,Bar,Café,Flea Market,Lounge,Asian Restaurant,Ice Cream Shop,South Indian Restaurant,Portuguese Restaurant,Plaza,5
2,Mexico City,Mexico,North America,19.432601,-99.133342,Mexican Restaurant,Art Museum,Museum,Arts & Crafts Store,Bar,Ice Cream Shop,Boutique,Bakery,Restaurant,Café,14
3,São Paulo,Brazil,South America,-23.550651,-46.633382,Brazilian Restaurant,Japanese Restaurant,Café,Bakery,Snack Place,Bookstore,Cosmetics Shop,Asian Restaurant,Art Gallery,Chinese Restaurant,9
4,Lagos,Nigeria,Africa,6.455057,3.394179,Lounge,African Restaurant,Bar,Café,Shopping Mall,Pizza Place,Hotel,Art Gallery,Italian Restaurant,Market,14


In [6]:
#Import Venue Count dataframe
obj = resource.Object(bucket_name=bucket, key='VenueCatCount.json').get()
df_cat = pd.read_json(io.BytesIO(obj['Body'].read()))
df_cat.sort_index(inplace=True)
df_cat.head()

,Metro,ATM,Acai House,Accessories Store,Afghan Restaurant,African Restaurant,Alsatian Restaurant,American Restaurant,Amphitheater,Antique Shop,...,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Yoshoku Restaurant,Zhejiang Restaurant,Zoo Exhibit
0,Ahmedabad,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Alexandria,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Ankara,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
3,Atlanta,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
4,Bandung,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
#How many metro areas are in each cluster?
df_count = df.loc[:,['Metropolitan','Cluster Labels']].groupby('Cluster Labels').count()
df_count

,Metropolitan
Cluster Labels,
0,1
1,12
2,1
3,1
4,21
5,8
6,1
7,6
8,1


In [8]:
#Look at the areas and top categories in each cluster containing more than 2 metro areas in the cluster
dftemp = df.loc[:,['Metropolitan','Cluster Labels']]
df_New = dftemp.join(df_cat.set_index('Metro'), on='Metropolitan')
dfR = df_New.groupby('Cluster Labels', as_index=False).sum()
for i in df_count[df_count['Metropolitan']>2].index:
    print('Cluster ',i)
    print('Top 10 venue categpries:')
    print(*dfR[dfR['Cluster Labels'] == i].drop('Cluster Labels',axis=1).sort_values(by=i, axis = 1, ascending=False).iloc[:,range(10)].columns,sep=', ')
    with pd.option_context('display.max_rows', None):
        display(df[df['Cluster Labels']==i].iloc[:,range(3)])

Cluster  1
Top 10 venue categpries:
Chinese Restaurant, Café, Bakery, Park, Supermarket, Restaurant, Italian Restaurant, Japanese Restaurant, Coffee Shop, Noodle House


,Metropolitan,Country,Continent
10,Chengdu,China,Asia
12,Shanghai,China,Asia
23,Jakarta,Indonesia,Asia
35,Rhine-Ruhr,Germany,Europe
48,Chūkyō (Nagoya),Japan,Asia
50,Bandung,Indonesia,Asia
52,Randstad,Netherlands,Europe
53,Busan,South Korea,Asia
67,Manila,Philippines,Asia
77,Berlin/Brandenburg,Germany,Europe


Cluster  4
Top 10 venue categpries:
Coffee Shop, Italian Restaurant, Plaza, Café, Cocktail Bar, Ice Cream Shop, Bakery, Park, Seafood Restaurant, Boutique


,Metropolitan,Country,Continent
15,Moscow,Russia,Europe
21,London,United Kingdom,Europe
27,Paris,France,Europe
34,Seoul,South Korea,Asia
41,Chicago,United States,North America
42,"Washington, D.C.",United States,North America
49,San Francisco,United States,North America
51,Boston,United States,North America
54,Milan,Italy,Europe
58,Riyadh,Saudi Arabia,Asia


Cluster  5
Top 10 venue categpries:
Indian Restaurant, Café, Fast Food Restaurant, Restaurant, Ice Cream Shop, Dessert Shop, Vegetarian / Vegan Restaurant, Bakery, Seafood Restaurant, Bar


...

,Metropolitan,Country,Continent
6,Cairo,Egypt,Africa
11,Dhaka,Bangladesh,Asia
17,Istanbul,Turkey,Europe
47,Bogotá,Colombia,South America
79,Munich,Germany,Europe
92,Sydney,Australia,Oceania
94,Alexandria,Egypt,Africa
99,Khartoum,Sudan,Africa


Cluster  14
Top 10 venue categpries:
Bar, Mexican Restaurant, Restaurant, Ice Cream Shop, Coffee Shop, Pizza Place, Café, Lounge, Park, Cocktail Bar


,Metropolitan,Country,Continent
2,Mexico City,Mexico,North America
4,Lagos,Nigeria,Africa
9,Los Angeles,United States,North America
25,Buenos Aires,Argentina,South America
36,Lahore,Pakistan,Asia
40,Lima,Peru,South America
55,Dallas-Fort Worth,United States,North America
66,Houston,United States,North America
69,Miami,United States,North America
71,Atlanta,United States,North America


Cluster  15
Top 10 venue categpries:
Shopping Mall, Coffee Shop, Café, Chinese Restaurant, Nightclub, Park, Bar, Noodle House, Asian Restaurant, Thai Restaurant


,Metropolitan,Country,Continent
13,Chongqing,China,Asia
18,Bangkok,Thailand,Asia
33,Nanjing,China,Asia
44,Johannesburg,South Africa,Africa
45,Guangzhou,China,Asia
46,Qingdao,China,Asia
89,Shenzhen,China,Asia


In [9]:
#Look at the areas in clusters containing 1 or 2 metro areas in the cluster
for i in df_count[df_count['Metropolitan']<=2].index:
    print('Cluster ',i)
    display(df[df['Cluster Labels']==i].iloc[:,range(3)])

Cluster  0


,Metropolitan,Country,Continent
30,Kinshasa,Democratic Republic of the Congo,Africa


Cluster  2


,Metropolitan,Country,Continent
16,Tianjin,China,Asia


Cluster  3


...

,Metropolitan,Country,Continent
26,Xi'an,China,Asia


Cluster  10


,Metropolitan,Country,Continent
14,Karachi,Pakistan,Asia
19,Kolkata,India,Asia


Cluster  11


,Metropolitan,Country,Continent
64,Yangon,Myanmar,Asia


In [10]:
#Create an interactive folium map with different color markers for each cluster
map_clusters = folium.Map(location=[0,0], zoom_start=2)

#set color scheme for the clusters
map_colors = [colors.rgb2hex(cm.tab20b(i)) for i in range(len(df_count.index)+1)]

# add markers to the map
markers_colors = []
for latitude, longitude, metro, country, cluster in zip(df['Latitude'], df['Longitude'], df['Metropolitan'], df['Country'], df['Cluster Labels']):
    label = folium.Popup(str(metro) + ', ' + str(country)+'- Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color=map_colors[cluster-1],
        fill=True,
        fill_color=map_colors[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters.save('map.html')
map_clusters

### View the interactive map
https://kathryndh.github.io/clustermap.html

In [11]:
#Save the map as an html file
obj = io.open('map.html','rb')
resource.Bucket(name=bucket).put_object(Key='map.html', Body=obj.read())
obj.close()
print('Saved')

Saved


<div style="text-align: left">
    <h2>
        <a href="https://nbviewer.jupyter.org/github/KathrynDH/IBMCapstoneFinalProject/blob/master/Cluster%20Metro%20Areas.ipynb">&larr; Part 4</a>
    </h2>
</div>